In [ ]:
import descarteslabs as dl
import os
from random import randint
from time import sleep

In [ ]:
def catalog_upload(year):
    
    # generate product ID and title from username
    user = os.environ.get('USER')
    
    product_id = '{}:{}:test'.format(
        dl.Auth().namespace,
        ''.join(user.lower().split(' '))
    )
    product_title = '{} Test'.format(user)
    
    # check if product already exists, remove if it does
    already_exists = list(filter(lambda p: p.get('id') == product_id, dl.Catalog().own_products()))
    if already_exists:
        print("Product already exists, removing existing one...")
        dl.Catalog().remove_product(product_id, cascade=True)
        print("...done")

    # define the product
    product_id = dl.Catalog().add_product(
            product_id,
            title=product_title,
            description='SAR image statistics (max/min/mean/median/std) from VV & VH backscatter'
            )['data']['id']
    print(product_id)

    # add band id
    bands=['vv', 'vh']
    passes = ['DESCENDING', 'ASCENDING', 'BOTH']
    stats = ['max', 'min', 'mean', 'median', 'std']

    for i, band in enumerate(bands):
        for p in passes:            
            for stat in stats:
                band_id = dl.Catalog().add_band(
                    product_id,
                    '{}_{}_{}'.format(band,stat,p),
                    jpx_layer=0,
                    srcfile=0,
                    srcband=i + 1,
                    nbits=8,
                    dtype='Byte',
                    nodata=0,
                    data_range=[0, 2**8 - 1],
                    type='spectral',
                    default_range=(0,255),
                    colormap_name='viridis',
                )['data']['id']
                print("Added band {}".format(band_id))
    print("Done! View your product here: https://catalog.descarteslabs.com/?/product/{}".format(product_id))

In [ ]:
catalog_upload(2017)